In [1]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tf.contrib.distributions as ds

from edward.models import Normal, Uniform

N = 300

# Model

X = tf.placeholder(tf.float32, [N, 2])
e = tf.placeholder(tf.float32, [N, 2])
Xh = Uniform(low=[0., 0.], high=[4200., 4200.])

m = ds.TransformedDistribution(
    distribution=ds.Normal(loc=0., scale=1.),
    bijector=ds.bijectors.Exp(),
    name="LogNormalTransformedDistribution")

sigma = Normal(loc=tf.zeros(1), scale=tf.ones(1))

l2diff = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(X, Xh)), reduction_indices=1))

sess = tf.session()
print(sess.run(l2diff))

mu = m / tf.maximum(l2diff, 240)

e1 = Normal(loc = mu, scale = sigma)
e2 = Normal(loc = mu, scale = sigma)

# Inference

qmu_loc = tf.Variable(tf.random_normal([]))
qmu_scale = tf.nn.softplus(tf.Variable(tf.random_normal([])))
qmu = Normal(loc=qmu_loc, scale=qmu_scale)

inference = ed.KLqp({mu: qmu}, data={x: x_data})
inference.run()

KeyboardInterrupt: 